In [32]:
from app.dataprocessing.data_handler import DataHandler
from app.dataprocessing.benchmark import plt_img, save_simple_img, demo_plt_img
from app.datastructures.datastructure_interface import get_ipyleaflet_bounds
from app.dataprocessing.benchmark import Stopwatch

from dotenv import load_dotenv
import os


#### Option 1: GEBCO’s Gridded Bathymetry Data
source: Local

Size: 7.5GB

In [22]:
data_handler = DataHandler()

data_handler.set_max_chunk_size(50)

file_path = "app/externalresources/datasets/GEBCO_2021.nc"
file_path = "app/externalresources/datasets/GEBCO_2022_sub_ice_topo.nc"

data_handler.set_local_netcdf_reader(file_path, 'KDTree')


print(f"Data structure in use: {data_handler.data_structure}")


initial_ds, bnds, node = data_handler.get_initial_ds()
#initial_ds.to_netcdf("panoplydemo.nc")
data_variable = "elevation"
img_format = 'png'

arr = initial_ds[data_variable].isel(
    {x: 0 for x in initial_ds.dims if x not in ["lat", "lon", "latitude", "longitude"]}
)
vmin_glob = arr.min().values
vmax_glob = arr.max().values

timer = Stopwatch()
timer.start('Plot inital render')
inital_render = plt_img(arr, vmin_glob, vmax_glob)
print(timer.stop())
bounds = get_ipyleaflet_bounds(initial_ds)

initial_ds

Finished 'Loading dataset' in 0.0054 seconds
Finished 'Creating data structure' in 0.5844 seconds
Data structure in use: KDTree with 256 leaf nodes of maximum size 50MB
Plot inital render took 12.8893 seconds


<xarray.Dataset>
Dimensions:    (lon: 7200, lat: 3600)
Coordinates:
  * lon        (lon) float64 -180.0 -179.9 -179.9 -179.8 ... 179.9 179.9 180.0
  * lat        (lat) float64 -90.0 -89.95 -89.9 -89.85 ... 89.8 89.85 89.9 89.95
Data variables:
    crs        |S1 ...
    elevation  (lat, lon) int16 ...
Attributes: (12/36)
    title:                           The GEBCO_2022 Grid - a continuous terra...
    summary:                         The GEBCO_2022 Grid is a continuous, glo...
    keywords:                        BATHYMETRY/SEAFLOOR TOPOGRAPHY, DIGITAL ...
    Conventions:                     CF-1.6, ACDD-1.3
    id:                              DOI: 10.5285/e0f0bb80-ab44-2739-e053-6c8...
    naming_authority:                https://dx.doi.org
    ...                              ...
    geospatial_vertical_units:       meters
    geospatial_vertical_resolution:  1.0
    geospatial_vertical_positive:    up
    identifier_product_doi:          DOI: 10.5285/e0f0bb80-ab44-2739-e053-6c8...
    references:                      DOI: 10.5285/e0f0bb80-ab44-2739-e053-6c8...
    node_offset:                     1.0

In [9]:
full_ds = data_handler.ds

tmp_arr = full_ds[data_variable].isel(
    {x: 0 for x in full_ds.dims if x not in ["lat", "lon", "latitude", "longitude"]}
)
timer = Stopwatch()
timer.start('Plot inital render')
bench_render = plt_img(tmp_arr, vmin_glob, vmax_glob)
print(timer.stop())


: 

: 

#### Option 2: GS17_ROV01_LokiCastle
source: Local

Size: 26.5 MB

In [2]:
data_handler = DataHandler()

data_handler.set_max_chunk_size(1)

file_path = 'app/externalresources/datasets/GS17_ROV01_LokiCastle_08072017_LLD_depthcorrected.nc'

data_handler.set_local_netcdf_reader(file_path)


print(f"Data structure in use: {data_handler.data_structure}")

initial_ds, bnds, node = data_handler.get_initial_ds()

data_variable = "z"
img_format = 'png'

arr = initial_ds[data_variable].isel(
    {x: 0 for x in initial_ds.dims if x not in ["lat", "lon", "latitude", "longitude"]}
)
vmin_glob = arr.min().values
vmax_glob = arr.max().values

inital_render = plt_img(arr, vmin_glob, vmax_glob)
bounds = get_ipyleaflet_bounds(initial_ds)

initial_ds

Finished 'Loading dataset' in 0.0482 seconds
Finished 'Creating data structure' in 2.6461 seconds
Data structure in use: QuadTree with 64 chunks at lowest level of max chunk size 1MB


<xarray.Dataset>
Dimensions:  (lon: 2091, lat: 535)
Coordinates:
  * lon      (lon) float64 8.146 8.147 8.147 8.147 ... 8.169 8.169 8.169 8.169
  * lat      (lat) float64 73.56 73.56 73.56 73.56 ... 73.57 73.57 73.57 73.57
Data variables:
    z        (lat, lon) float32 ...
Attributes:
    Conventions:  COARDS, CF-1.5
    title:        
    history:      nearneighbor GS17_ROV01_LokiCastle_08072017_LLD_depthcorrec...
    description:  
    GMT_version:  5.4.1 (r18208) [64-bit]

#### Option 3: GLOBAL_ANALYSIS_FORECAST_PHY_001_024
source: Remote, OPeNDAP Copernicus

Size: unknown

In [21]:
data_handler = DataHandler()
data_handler.set_max_chunk_size(50)

load_dotenv()
USERNAME = os.environ.get("CMEMS_CAS_USERNAME")
PASSWORD = os.environ.get("CMEMS_CAS_PASSWORD")

dataset_url = 'https://nrt.cmems-du.eu/thredds/dodsC/global-analysis-forecast-phy-001-024-monthly'
cas_url = "https://cmems-cas.cls.fr/cas/login"

constraints = {'time' : 19, 'depth':0}


data_handler.set_opendap_cas(cas_url, dataset_url, USERNAME, PASSWORD, constraints=constraints)


# Show the auto-selected data structure
print(f"Data structure in use: {data_handler.data_structure}")

Data structure in use: QuadTree with 16 chunks at lowest level of max chunk size 50MB


In [23]:
data_handler.ds

<xarray.Dataset>
Dimensions:    (latitude: 2041, longitude: 4320)
Coordinates:
    depth      float32 0.494
  * latitude   (latitude) float32 -80.0 -79.92 -79.83 ... 89.83 89.92 90.0
  * longitude  (longitude) float32 -180.0 -179.9 -179.8 ... 179.8 179.8 179.9
    time       datetime64[ns] 2021-08-16T12:00:00
Data variables:
    mlotst     (latitude, longitude) float32 ...
    zos        (latitude, longitude) float32 ...
    bottomT    (latitude, longitude) float32 ...
    sithick    (latitude, longitude) float32 ...
    siconc     (latitude, longitude) float32 ...
    usi        (latitude, longitude) float32 ...
    vsi        (latitude, longitude) float32 ...
    thetao     (latitude, longitude) float32 ...
    so         (latitude, longitude) float32 ...
    uo         (latitude, longitude) float32 ...
    vo         (latitude, longitude) float32 ...
Attributes: (12/14)
    title:                         Monthly mean fields for product GLOBAL_ANA...
    references:                    http://marine.copernicus.eu
    credit:                        E.U. Copernicus Marine Service Information...
    licence:                       http://marine.copernicus.eu/services-portf...
    contact:                       servicedesk.cmems@mercator-ocean.eu
    producer:                      CMEMS - Global Monitoring and Forecasting ...
    ...                            ...
    area:                          GLOBAL
    product:                       GLOBAL_ANALYSIS_FORECAST_PHY_001_024
    dataset:                       global-analysis-forecast-phy-001-024-monthly
    source:                        MERCATOR PSY4QV3R1
    product_user_manual:           http://marine.copernicus.eu/documents/PUM/...
    quality_information_document:  http://marine.copernicus.eu/documents/QUID...

In [27]:
initial_ds, bnds, node = data_handler.get_initial_ds()

data_variable = "thetao"
img_format = 'png'

arr = initial_ds[data_variable].isel(
    {x: 0 for x in initial_ds.dims if x not in ["lat", "lon", "latitude", "longitude"]}
)
vmin_glob = arr.min().values
vmax_glob = arr.max().values

demo = demo_plt_img(arr, vmin_glob, vmax_glob)
inital_render = plt_img(arr, vmin_glob, vmax_glob)
bounds = get_ipyleaflet_bounds(initial_ds)

print(vmin_glob)
print(vmax_glob)

initial_ds

-2.3312778
35.64742


<xarray.Dataset>
Dimensions:    (latitude: 681, longitude: 1440)
Coordinates:
    depth      float32 0.494
  * latitude   (latitude) float32 -80.0 -79.75 -79.5 -79.25 ... 89.5 89.75 90.0
  * longitude  (longitude) float32 -180.0 -179.8 -179.5 ... 179.2 179.5 179.8
    time       datetime64[ns] 2021-08-16T12:00:00
Data variables:
    mlotst     (latitude, longitude) float32 ...
    zos        (latitude, longitude) float32 ...
    bottomT    (latitude, longitude) float32 ...
    sithick    (latitude, longitude) float32 ...
    siconc     (latitude, longitude) float32 ...
    usi        (latitude, longitude) float32 ...
    vsi        (latitude, longitude) float32 ...
    thetao     (latitude, longitude) float32 ...
    so         (latitude, longitude) float32 ...
    uo         (latitude, longitude) float32 ...
    vo         (latitude, longitude) float32 ...
Attributes: (12/14)
    title:                         Monthly mean fields for product GLOBAL_ANA...
    references:                    http://marine.copernicus.eu
    credit:                        E.U. Copernicus Marine Service Information...
    licence:                       http://marine.copernicus.eu/services-portf...
    contact:                       servicedesk.cmems@mercator-ocean.eu
    producer:                      CMEMS - Global Monitoring and Forecasting ...
    ...                            ...
    area:                          GLOBAL
    product:                       GLOBAL_ANALYSIS_FORECAST_PHY_001_024
    dataset:                       global-analysis-forecast-phy-001-024-monthly
    source:                        MERCATOR PSY4QV3R1
    product_user_manual:           http://marine.copernicus.eu/documents/PUM/...
    quality_information_document:  http://marine.copernicus.eu/documents/QUID...

#### Option 4: Baltic Sea Physics Reanalysis (SUBSET, time index 0-30)
source: Local

Size: 5.2GB

In [71]:
data_handler = DataHandler()

data_handler.set_max_chunk_size(50)

file_path = "app/externalresources/datasets/BALTICSEA_REANALYSIS_PHY_003_011_slice.nc"

constraints = {'depth':0}

data_handler.set_local_netcdf_reader(file_path, constraints=constraints, struct='KDTree')


print(f"Data structure in use: {data_handler.data_structure}")

initial_ds, bnds, node = data_handler.get_initial_ds()

data_variable = "thetao"  # potential temperature
img_format = "png"

arr = initial_ds[data_variable].isel(
    {x: 0 for x in initial_ds.dims if x not in ["lat", "lon", "latitude", "longitude"]}
)
vmin_glob = arr.min().values
vmax_glob = arr.max().values

inital_render = plt_img(arr, vmin_glob, vmax_glob)
bounds = get_ipyleaflet_bounds(initial_ds)

initial_ds


Finished 'Loading dataset' in 0.0257 seconds
Finished 'Creating data structure' in 0.4089 seconds
Data structure in use: KDTree with 128 leaf nodes of maximum size 50MB


<xarray.Dataset>
Dimensions:    (latitude: 105, longitude: 77, time: 6)
Coordinates:
    depth      float32 1.501
  * latitude   (latitude) float32 48.49 48.66 48.83 48.99 ... 65.49 65.66 65.82
  * longitude  (longitude) float32 9.014 9.292 9.569 9.847 ... 29.57 29.85 30.12
  * time       (time) datetime64[ns] 1993-01-01T12:00:00 ... 1993-01-26T12:00:00
Data variables:
    sob        (time, latitude, longitude) float32 ...
    bottomT    (time, latitude, longitude) float32 ...
    mlotst     (time, latitude, longitude) float32 ...
    so         (time, latitude, longitude) float32 ...
    thetao     (time, latitude, longitude) float32 ...
    uo         (time, latitude, longitude) float32 ...
    vo         (time, latitude, longitude) float32 ...
Attributes: (12/23)
    references:             http://www.smhi.se
    institution:            Swedish Meterological and Hydrological Institute
    history:                See source and creation_date attributees
    Conventions:            CF-1.5
    contact:                servicedesk_cmems@mercator-ocean.eu
    comment:                Provided by SMHI as a Copernicus Marine Environme...
    ...                     ...
    creation_date:          2021-11-09 UTC
    bullentin_date:         20201231
    start_date:             2020-12-31 UTC
    stop_date:              2020-12-31 UTC
    start_time:             00:00 UTC
    stop_time:              00:00 UTC

In [34]:
d = data_handler.data_structure.root.bounds
print(d)

((<xarray.DataArray 'latitude' ()>
array(48.4917, dtype=float32)
Coordinates:
    depth    float32 1.501, <xarray.DataArray 'latitude' ()>
array(65.82475, dtype=float32)
Coordinates:
    depth    float32 1.501), (<xarray.DataArray 'longitude' ()>
array(9.013755, dtype=float32)
Coordinates:
    depth    float32 1.501, <xarray.DataArray 'longitude' ()>
array(30.124655, dtype=float32)
Coordinates:
    depth    float32 1.501), (<xarray.DataArray 'time' ()>
array('1993-01-01T12:00:00.000000000', dtype='datetime64[ns]')
Coordinates:
    depth    float32 1.501, <xarray.DataArray 'time' ()>
array('1993-01-26T12:00:00.000000000', dtype='datetime64[ns]')
Coordinates:
    depth    float32 1.501))


In [72]:
import numpy as np
import datetime

s = ((50, 60), (15,25), (np.datetime64('1993-01-20'), np.datetime64('1993-01-20')))
#s = ((-74.09, -3.78), (-155.45, -56.49), (datetime.datetime(2000, 1, 1), datetime.datetime(2012, 1, 26)))
file_name, bounds, node = data_handler.request_data_netcdf(
                    s, return_xr_chunk=True
                )
node.ds

KeyboardInterrupt: 

In [70]:
print(str(bounds[2][0].values)[:10])

1993-01-01


In [58]:
print(bounds[0][0].values)

48.4917


#### Render map

In [23]:
from ipyleaflet import basemaps, Map, ImageOverlay, projections
import ipywidgets
import numpy as np

from app.datastructures.datastructure_interface import spatial_resolution


def print_bounds(bounds):
    sw, ne = bounds
    s, w = sw
    n, e = ne
    return f"SW: ({s:.2f}, {w:.2f}) NE: ({n:.2f}, {e:.2f})"


button = ipywidgets.Button(description="Render")
button2 = ipywidgets.Button(description="Clear Layers")
button3 = ipywidgets.Button(description="Render, new colors")
button4 = ipywidgets.Button(description="List Cache")

full_ds = data_handler.get_full_xr_ds()


def create_dim_sliders(ds):
    sliders = []
    for d in ds.dims:
        if d not in ["lat", "lon", "latitude", "longitude"]:
            s = ipywidgets.SelectionSlider(
                options=list(ds[d].values),
                value=ds[d][0],
                description=f"{d}: ",
                disabled=False,
                continuous_update=False,
                orientation="horizontal",
                readout=True,
                layout={"width": "700px"},
                # style = {'width':'initial'}
            )
            sliders.append(s)
    return sliders


dim_sliders = create_dim_sliders(full_ds)


m = Map(crs=projections.EPSG4326)
m.clear_layers()  # remove default map. No default map supports EPSG4326 projection

image = ImageOverlay(url=inital_render, bounds=bounds, opacity=1)
m.add_layer(image)

m.fit_bounds(bounds)
m.max_zoom = 60


output = ipywidgets.Output()
with output:
    #print(f"Initial render at {data_handler.get_node_resolution(node):.2f}% resolution")
    print(
        f"Dataset size: {data_handler.data_source.file_size_MB:.2f} MB, Max chunk size: {data_handler.max_chunk_size} MB"
    )
    print(f"Bounds: {print_bounds(bounds)}")
    print(f"-----------------------------------------------------")
# display(button, button2, m, output)

display(ipywidgets.HBox((button, button3, button2, button4)), *dim_sliders, m, output)


def handle_map_interaction(**kwargs):
    if kwargs.get("type") == "mouseup":
        with output:
            print(m.bounds)


def render(reset_color_scale=False):
    with output:
        timer = Stopwatch()

        print(f"Query bounds: {print_bounds(m.bounds)}")

        timer.start("Fetch netCDF chunk")
        sw_ne = m.bounds

        query_bounds = ((sw_ne[0][0], sw_ne[1][0]), (sw_ne[0][1], sw_ne[1][1]))
        print(query_bounds)
        # (np.datetime64("2021-09-23"), np.datetime64("2021-09-25")),

        file_name, bounds, node = data_handler.request_data_netcdf(
            query_bounds, return_xr_chunk=True
        )
        fetch_time = timer.stop()

        timer.start("Rendering")
        respons_bounds_sw_ne = (
            (bounds[0][0], bounds[1][0]),
            (bounds[0][1], bounds[1][1]),
        )

        if reset_color_scale:
            vmin = arr.min().values
            vmax = arr.max().values
        else:
            vmin = vmin_glob
            vmax = vmax_glob

        render = save_simple_img(node.ds, data_variable, vmin, vmax, img_format)

        # change opacity on old layers
        for l in m.layers:
            if isinstance(l, ImageOverlay):
                l.opacity = 0.8

        image = ImageOverlay(url=render, bounds=respons_bounds_sw_ne, opacity=1)
        m.add_layer(image)
        render_time = timer.stop()

        print(f"Chunk bounds: {print_bounds(respons_bounds_sw_ne)}")
        #print(
        #    f"Chunk resolution: {data_handler.get_node_resolution(node):.2f}%, File size: {data_handler.get_file_size_MB(file_name):.2f} MB"
       # )
        print(f"Spatial resolution: {spatial_resolution(node.ds)}")
        print(f"{fetch_time}")
        print(f"{render_time}")
        print(f"-----------------------------------------------------")


def clear_map(event):
    m.clear_layers()


def update_map(event):
    render(reset_color_scale=False)


def reset_vmin_vmax(event):
    render(reset_color_scale=True)

def print_cache(event):
    with output:
        cache = data_handler.get_cache()
        for idx, c in enumerate(cache):
            print(f'Chunk {idx}:')
            print(c)
        print(f"-----------------------------------------------------")

button.on_click(update_map)
button2.on_click(clear_map)
button3.on_click(reset_vmin_vmax)
button4.on_click(print_cache)
# m.on_interaction(handle_map_interaction)


Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

Output()